In [ ]:
"""
Pairwise evaluation. Simulated vs. OTA datasets.
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import funcs
import jan
import coral
import star
import mcd
import dann
import base
import plots
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Load testbed data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "/home/ash/ic3/testbed_da/data"

# Classes in loaded npy files
class_subset = ["bpsk", "qpsk", "16qam", "16apsk"]

# simulated data
X_sim = np.load(file_path + "/sim_X.npy")
Y_sim = np.load(file_path + "/sim_Y.npy")

# over the air data
X_ota = np.load(file_path + "/ota_X.npy")
Y_ota = np.load(file_path + "/ota_Y.npy")

z_val = 10
sim_acc = []
ota_acc = []

for i in range(5):
    print(f'SNR level: {z_val}')
    # Filter for SNR level
    source_mask = (Y_sim[:, 1] == z_val+100)
    X_s = X_sim[source_mask]
    Y_s = Y_sim[source_mask]
    Y_s = Y_s[:,0]

    source_mask = (Y_ota[:, 1] == z_val)
    X_t = X_ota[source_mask]
    Y_t = Y_ota[source_mask]
    Y_t = Y_t[:,0]
    
    S_train_loader, S_val_loader = funcs.create_loader(X_s, Y_s, permute=False)
    T_train_loader, T_val_loader = funcs.create_loader(X_t, Y_t, permute=False)

    print('BASE')
    s_base, t_base = base.Base(model_cls=base.CLDNN, device=device, S_train_loader=S_train_loader, 
                    S_val_loader=S_val_loader, T_val_loader=T_val_loader, class_subset=class_subset, 
                    n_classes=len(class_subset), lr=0.005, n_epochs=25, n_runs=1).run()
    torch.cuda.empty_cache()
    sim_acc.append(s_base)
    ota_acc.append(t_base)

    z_val += 2


In [ ]:



x = np.arange(1, 6)

plt.figure(figsize=(10, 6))
plt.plot(x, sim_acc, marker='o', linestyle='-', label='Sim')
plt.plot(x, ota_acc, marker='s', linestyle='-', label='Ota')


plt.xlabel('SNR (dB)')
plt.ylabel('Acc (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# simulated data
X_sim = np.load(file_path + "/sim_X.npy")
Y_sim = np.load(file_path + "/sim_Y.npy")

# over the air data
X_ota = np.load(file_path + "/ota_X.npy")
Y_ota = np.load(file_path + "/ota_Y.npy")

z_val = 10
sim_acc = []
ota_acc = []

for i in range(5):
    source_mask = (Y_sim[:, 1] == z_val+100)
    X_s = X_sim[source_mask]
    Y_s = Y_sim[source_mask]
    Y_s = Y_s[:,0]

    source_mask = (Y_ota[:, 1] == z_val)
    X_t = X_ota[source_mask]
    Y_t = Y_ota[source_mask]
    Y_t = Y_t[:,0]

    S_train_loader, S_val_loader = funcs.create_loader(X_s, Y_s, permute=False)
    T_train_loader, T_val_loader = funcs.create_loader(X_t, Y_t, permute=False)
    
    print(f'SNR level: {z_val}')
    print('DANN')
    s_base, t_base = dann.DAN(dann.DANN, FA=dann.CLDNN_FA, LP=dann.CLDNN_LP, DC=dann.CLDNN_DC,
                      device=device, S_train_loader=S_train_loader, S_val_loader=S_val_loader,
                      T_train_loader=T_train_loader, T_val_loader=T_val_loader,
                      class_subset=class_subset, n_classes=len(class_subset), lr=0.005,
                      n_epochs=50, n_runs=5).run()
    torch.cuda.empty_cache()
    sim_acc.append(s_base)
    ota_acc.append(t_base)

    z_val += 2
    

In [ ]:


x = np.arange(1, 6)

plt.figure(figsize=(10, 6))
plt.plot(x, sim_acc, marker='o', linestyle='-', label='Sim')
plt.plot(x, ota_acc, marker='s', linestyle='-', label='Ota')


plt.xlabel('SNR (dB)')
plt.ylabel('Acc (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# simulated data
X_sim = np.load(file_path + "/sim_X.npy")
Y_sim = np.load(file_path + "/sim_Y.npy")

# over the air data
X_ota = np.load(file_path + "/ota_X.npy")
Y_ota = np.load(file_path + "/ota_Y.npy")

z_val = 10
sim_acc = []
ota_acc = []

for i in range(5):
    source_mask = (Y_sim[:, 1] == z_val+100)
    X_s = X_sim[source_mask]
    Y_s = Y_sim[source_mask]
    Y_s = Y_s[:,0]

    source_mask = (Y_ota[:, 1] == z_val)
    X_t = X_ota[source_mask]
    Y_t = Y_ota[source_mask]
    Y_t = Y_t[:,0]

    S_train_loader, S_val_loader = funcs.create_loader(X_s, Y_s, permute=False)
    T_train_loader, T_val_loader = funcs.create_loader(X_t, Y_t, permute=False)
 
    print(f'SNR level: {z_val}')
    print('STAR')
    s_base, t_base = star.Star(G=star.CLDNN_G, C=star.CLDNN_C, device=device, S_train_loader=S_train_loader, S_val_loader=S_val_loader,  
                   T_train_loader=T_train_loader, T_val_loader=T_val_loader, class_subset=class_subset,
                   n_classes=len(class_subset), lr=0.005, n_epochs=50, n_runs=5, patience=5).run()
    torch.cuda.empty_cache()
    sim_acc.append(s_base)
    ota_acc.append(t_base)

    z_val += 2
    


In [ ]:


x = np.arange(1, 6)

plt.figure(figsize=(10, 6))
plt.plot(x, sim_acc, marker='o', linestyle='-', label='Sim')
plt.plot(x, ota_acc, marker='s', linestyle='-', label='Ota')


plt.xlabel('SNR (dB)')
plt.ylabel('Acc (%)')
plt.legend()
plt.grid(True)
plt.show()